In [18]:
import pandas as pd

In [19]:
user_streaming = pd.read_excel("YOUR_PATH_TO\\WebsiteUserStreaming.xlsx", header=0)

In [20]:
user_streaming

,userid,time,source,event
0,1111,2020-01-01,blog,null
1,1111,2020-01-02,organic,null
2,2222,2020-01-01,ppc,null
3,2222,2020-01-02,blog,null
4,2222,2020-01-03,organic,CONVERSION
5,6666,2020-01-01,ppc,null


In [58]:
conversions = user_streaming['event'][user_streaming['event'] == 'CONVERSION'].count()
if(conversions == 0):
    print('Nothing to count, wait for at least 1 conversion')


1

In [21]:
chains = []
for i, userid in enumerate(user_streaming['userid']):
    if (i == 0):
        user_path = user_streaming[user_streaming['userid'] == userid]
        chain = 'START'
        for path in user_path['source']:
            chain += '-' + path
        chain += '-' + user_path['event'].iloc[-1]
        chains.append(chain)
        prev_user = userid
    else:
        if(userid == prev_user):
            continue;
        else:
            user_path = user_streaming[user_streaming['userid'] == userid]
            chain = 'START'
            for path in user_path['source']:
                chain += '-' + path
            chain += '-' + user_path['event'].iloc[-1]
            chains.append(chain)
            prev_user = userid

In [22]:
chains_transitions = []
for chain in chains:
    transitions = chain.split('-')
    for cur, nxt in zip (transitions, transitions[1:] ):
        chains_transitions.append(cur+'-'+nxt)     

In [23]:
chains_transitions.sort()
chains_transitions

['START-blog',
 'START-ppc',
 'START-ppc',
 'blog-organic',
 'blog-organic',
 'organic-CONVERSION',
 'organic-null',
 'ppc-blog',
 'ppc-null']

In [24]:
channels_from = []
channels_to = []
num_transitions = []
total_from = []
for i, chain in enumerate(chains_transitions):
    num = chains_transitions.count(chain)
    num_transitions.append(num)
    channel_from = chain[:chain.index('-')]
    channels_from.append(channel_from)
    channel_to = chain[chain.index('-')+1:]
    channels_to.append(channel_to)
    
for channel in channels_from:
    total_from.append(channels_from.count(channel))
    
transition_matrix = pd.DataFrame(list(zip(channels_from, channels_to, num_transitions, total_from)), 
                                 columns =['channels_from', 'channels_to', 'num_transitions', 'total_from']) 
transition_matrix.drop_duplicates(subset=None, keep='first', inplace=True)
transition_matrix['probability'] = round(transition_matrix['num_transitions']/transition_matrix['total_from'], 2)
transition_matrix['from-to'] = transition_matrix['channels_from'] + '-' + transition_matrix['channels_to']

In [25]:
transition_matrix

,channels_from,channels_to,num_transitions,total_from,probability,from-to
0,START,blog,1,3,0.33,START-blog
1,START,ppc,2,3,0.67,START-ppc
3,blog,organic,2,2,1.00,blog-organic
5,organic,CONVERSION,1,2,0.50,organic-CONVERSION
6,organic,null,1,2,0.50,organic-null
7,ppc,blog,1,2,0.50,ppc-blog
8,ppc,null,1,2,0.50,ppc-null


In [26]:
def rec_seq(keyword, string, channel_list, end='-CONVERSION'):
    successful_channels = transition_matrix['channels_from'].loc[transition_matrix['channels_to'].isin([keyword])]
    for channel in successful_channels:
        if(channel == 'START'):
            for i in reversed(channel_list):
                string += '-' + i
            new_string = string + end
            successful_chains.append(new_string)
            string = 'START'
        else:
            channel_list.append(channel)
            rec_seq(channel, string, channel_list)

In [27]:
start = 'START'
channel_list = []
successful_chains = []
success = 'CONVERSION'
rec_seq(success, start, channel_list)

In [28]:
successful_chains

['START-blog-organic-CONVERSION', 'START-ppc-blog-organic-CONVERSION']

In [29]:
model_probability = []
for sequence in successful_chains:
    seq_probability = []
    for chain in transition_matrix['from-to']:
        if(sequence.__contains__(chain)):
            value = transition_matrix['probability'].loc[transition_matrix['from-to'].isin([chain])]
            value = value.values.tolist()
            value = value[0]
            seq_probability.append(value)
        else:
            continue
    print(seq_probability)
    result = 1
    for prob in seq_probability: 
         result = result * prob
    model_probability.append(result)
print(model_probability)
result = 0
for prob in model_probability:
    result = result + prob
model_probability = result    

[0.33, 1.0, 0.5]
[0.67, 1.0, 0.5, 0.5]
[0.165, 0.1675]


In [30]:
model_probability

0.3325

In [63]:
re_model_probability = []
re_sources = {}
sources = list(user_streaming['source'].drop_duplicates(inplace=False))
for source in sources:
    re_sources[source] = 0
    involve = 0
    for sequence in successful_chains:
        if (sequence.__contains__(source)):
            involve += 1
            if(involve == len(successful_chains)):
                re_sources[source] = 1
            continue
        else:
            seq_probability = []
            for chain in transition_matrix['from-to']:
                if(sequence.__contains__(chain)):
                    value = transition_matrix['probability'].loc[transition_matrix['from-to'].isin([chain])]
                    value = value.values.tolist()
                    value = value[0]
                    seq_probability.append(value)
                else:
                    continue
            result = 1
            for prob in seq_probability: 
                 result = result * prob
            re_model_probability.append(result)
    if (re_sources[source] == 1):
        continue
    else:
        result = 0
        for prob in re_model_probability:
            result = result + prob
        re_model_probability = result
        re_sources[source] = round(re_model_probability/model_probability, 2)
re_sources

{'blog': 1, 'organic': 1, 'ppc': 0.5}

In [80]:
num_cnv = {}
for source, coef_norm in re_sources.items():
    coef_sum = 0
    for values in re_sources.values():
        coef_sum += values
    num_cnv[source] = round(float(coef_norm/coef_sum*conversions), 2)
num_cnv

{'blog': 0.4, 'organic': 0.4, 'ppc': 0.2}

In [88]:
table_for_report = pd.DataFrame(num_cnv, index=[0])
table_for_report

,blog,organic,ppc
0,0.4,0.4,0.2
